In [7]:
# fixing PKLm
import numpy as np
a1 = np.empty((26))
a11 = np.empty([7])
b1 = np.empty((7,26))
c1 = np.empty((26,7))
vx = a1 + (a11.dot(b1))

t3 = vx + (b1)
t4 = c1 + t3

print('vx: ', vx.shape, t3.shape, t4.shape)

ValueError: operands could not be broadcast together with shapes (26,7) (7,26) 

In [2]:
import numpy as np
x = np.empty((7,7))
y = np.empty((7, 7))
z = np.empty([7])
zz = x.dot(y-z)
print(zz.shape)

# zz = np.zeros_like(x)
# print(zz.shape)

(7, 7)


In [23]:
import scipy as sp
q = -5*np.eye(5)
w,v = np.linalg.eigh(x)
print('w vals: ', w)
# print('w: ', np.linalg.eigh(x)[0])
# print('v vectors: ', v)wsort = np.amin(np.sort(w))
qchol = np.linalg.inv(q)
print(qchol)

('w vals: ', array([ -9.71197146e-310,  -8.15561661e-310,  -1.96489224e-313,
         1.85570289e-313,   2.80834431e-310,   1.83343715e-309,
         3.14891619e-309]))
[[-0.2 -0.  -0.  -0.  -0. ]
 [-0.  -0.2 -0.  -0.  -0. ]
 [-0.  -0.  -0.2 -0.  -0. ]
 [-0.  -0.  -0.  -0.2 -0. ]
 [-0.  -0.  -0.  -0.  -0.2]]


In [12]:
1400-490-974.94

-64.94000000000005

In [27]:
import numpy as np
a = np.arange(100).reshape(10,10)
b = np.arange(25, 50).reshape(5,5)

idx_x = slice(3)
idx_u = slice(3, 6)
idx_v = slice(6, 9)

c = a[idx_x, idx_x]
d = a[idx_x, idx_u]
e = a[idx_x, idx_v]
print(a)
print('c:\n',c)

print('d:\n', d)

print('e:\n', e)

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87 88 89]
 [90 91 92 93 94 95 96 97 98 99]]
c:
 [[ 0  1  2]
 [10 11 12]
 [20 21 22]]
d:
 [[ 3  4  5]
 [13 14 15]
 [23 24 25]]
e:
 [[ 6  7  8]
 [16 17 18]
 [26 27 28]]


In [44]:
dX, dU, dV = 26, 7, 7
dt = .01
gains = 1.0/np.array([3.09, 1.08, 0.393, 0.674, 0.111, 0.152, 0.098])
Fd = np.vstack([
    np.hstack([
        np.eye(dU), dt * np.eye(dU), np.zeros((dU, dX - dU*2)),
        dt ** 2 * np.diag(gains)
    ]),
    np.hstack([
        np.zeros((dU, dU)), np.eye(dU), np.zeros((dU, dX - dU*2)),
        dt * np.diag(gains)
    ]),
    np.zeros((dX - dU*2, dX+dU))
])
print(Fd.shape)
# print(Fd)
Ltt = np.hstack([
         np.ones(dU),
         np.ones(dU),
         np.zeros(dX - dU*2), np.ones(dU)])
print(Ltt.shape)
lt = Ltt.dot(np.r_([    
        np.concatenate([np.zeros(7), np.zeros(7)])
]) )
print(lt.shape)

(26, 33)
(33,)


TypeError: 'RClass' object is not callable

In [49]:
a = np.concatenate([np.zeros(7), np.zeros(7)])
b = np.r_[a, np.zeros(7)]
print(Ltt.dot(b).shape)

ValueError: shapes (33,) and (21,) not aligned: 33 (dim 0) != 21 (dim 0)

In [54]:
a = np.empty([66]).dot(np.empty([66]))
print(a)

nan
